### Cargando librerias

In [1]:
import pandas as pd
from scipy import ndimage
import os

In [7]:
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

In [10]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

### Creando funciones

In [72]:
def getSkewAngle(cvImage) -> float:
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=5)

    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)

    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle

In [80]:
def limpieza_imagen(imagen):
    gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (1, 1), 0)
    ret,thresh = cv2.threshold(blur, 127,255,cv2.THRESH_TOZERO)
    kernel = np.ones((3,3), np.uint8)
    img_erosion = cv2.erode(thresh, kernel, iterations=1)
    rotated = ndimage.rotate(img, -1*getSkewAngle(img))
    return rotated

### Leyendo pdfs

In [2]:
path = '/Users/lorozcoceron/Downloads/pdfs'

In [3]:
contenido = os.listdir(path)

In [17]:
pdf = [x for x in contenido if ".pdf" in x]

In [19]:
len(pdf)

32

In [14]:
from pdf2image import convert_from_path

### Convirtiendo pdf a imagenes

In [29]:
from pdf2image import convert_from_path
 
for i in range(len(pdf)):
    images = convert_from_path(path+"/"+pdf[i])
    images[0].save(path+"/"+pdf[i][:-4]+".jpg")

In [30]:
contenido = os.listdir(path)

In [5]:
images_filename = [x for x in contenido if ".jpg" in x]

### Procesando imagenes, extrayendo texto, guardando archivos txt

In [81]:
for i in range(len(images_filename)):
    img = cv2.imread(path+"/"+images_filename[i])
    img_pross = limpieza_imagen(img)
    texto = pytesseract.image_to_string(img, lang='spa')
    new_file = open(path+"/"+images_filename[i][:-4] + ".txt", "w+")
    new_file.write(texto)
    new_file.close()
